# Fiddler LangGraph SDK: Advanced Patterns and Production Observability

This notebook demonstrates advanced usage patterns for the Fiddler LangGraph SDK, focusing on production-ready agentic applications with comprehensive observability.

## What You'll Learn

- **Complex architectural patterns** for multi-agent systems
- **Advanced monitoring techniques** with custom metrics and context tracking
- **Performance optimization** strategies for production workloads
- **Error handling and resilience** patterns for reliable agents
- **Security and compliance** considerations for enterprise deployments

## Prerequisites

- Completed the [Fiddler LangGraph SDK Quick Start Guide](../quick-start-guide.md)
- Python 3.10+ environment
- OpenAI API key
- Fiddler instance access with API key and application ID

## Environment Setup

First, let's install dependencies and set up our environment:


In [ ]:
# Install required packages
# %pip install fiddler-langgraph langchain-openai langchain-chroma langgraph python-dotenv

# For beta participant installations, use:
%pip install --index-url https://test.pypi.org/simple/ --extra-index-url https://pypi.org/simple fiddler-langgraph==0.1.0a14
# %pip install langchain-openai langchain-chroma langgraph python-dotenv
%pip install langchain-openai langchain-chroma "langgraph>=0.5.0,<=0.5.2" python-dotenv

### Environment Configuration

Create a `.env` file in your working directory with your credentials.

```text
# FiddlerClient connection details
FIDDLER_URL=https://your-instance.fiddler.ai
FIDDLER_APPLICATION_ID=your-genai-application-id
FIDDLER_API_KEY=your-fiddler-access-token
```

In [ ]:
import os
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# Verify required environment variables
required_vars = [
    "FIDDLER_API_KEY",
    "FIDDLER_APPLICATION_ID", 
    "FIDDLER_URL",
]

missing_vars = [var for var in required_vars if not os.getenv(var)]
if missing_vars:
    print(f"❌ Missing environment variables: {missing_vars}")
    print("Please create a .env file with the required credentials.")
else:
    print("✅ All environment variables configured")
    print(f"Fiddler URL: {os.getenv('FIDDLER_URL')}")
    print(f"Application ID: {os.getenv('FIDDLER_APPLICATION_ID')}")

## Part 1: Advanced SDK Configuration and Performance Optimization

Let's start with production-ready configuration patterns that optimize performance and resource usage.

* Key concepts: Environment-specific configs, span limits, sampling, compression
* Why it matters: Production vs development trade-offs
* Learning value: Understanding performance optimization strategies

In [ ]:
import os
import logging
from uuid import uuid4
from typing import Any, Dict, List, Optional

from opentelemetry.sdk.trace import SpanLimits, sampling
from opentelemetry.exporter.otlp.proto.http.trace_exporter import Compression

from fiddler_langgraph import FiddlerClient

# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

def create_optimized_client(environment: str = "development") -> FiddlerClient:
    """
    Create environment-specific Fiddler client with optimized configuration.
    
    Args:
        environment: Target environment (development, staging, production)
        
    Returns:
        Configured FiddlerClient instance
    """
    
    if environment == "production":
        # Production: Optimize for minimal overhead
        os.environ.update({
            'OTEL_BSP_MAX_QUEUE_SIZE': '2000',
            'OTEL_BSP_SCHEDULE_DELAY_MILLIS': '100',
            'OTEL_BSP_MAX_EXPORT_BATCH_SIZE': '200',
            'OTEL_BSP_EXPORT_TIMEOUT': '10000',
        })
        
        span_limits = SpanLimits(
            max_events=32,
            max_links=16,
            max_span_attributes=64,
            max_event_attributes=32,
            max_link_attributes=16,
            max_span_attribute_length=2048,
        )
        
        sampler = sampling.TraceIdRatioBased(0.05)
        
        return FiddlerClient(
            api_key=os.getenv("FIDDLER_API_KEY"),
            application_id=os.getenv("FIDDLER_APPLICATION_ID"),
            url=os.getenv("FIDDLER_URL"),
            console_tracer=False,
            span_limits=span_limits,
            sampler=sampler,
            compression=Compression.Gzip,
        )
    else:
        # Development: Full observability for debugging
        return FiddlerClient(
            api_key=os.getenv("FIDDLER_API_KEY"),
            application_id=os.getenv("FIDDLER_APPLICATION_ID"),
            url=os.getenv("FIDDLER_URL"),
            console_tracer=True,
            compression=Compression.NoCompression,
        )

# Create our optimized client
environment = os.getenv("ENVIRONMENT", "development")
fdl_client = create_optimized_client(environment)

print(f"✅ Created {environment} client with optimized configuration")
print(f"Console tracing: {'Enabled' if environment == 'development' else 'Disabled'}")
print(f"Compression: {'Disabled (dev)' if environment == 'development' else 'Gzip enabled'}")

## Part 2: Multi-Agent System with Observability

Let's build a travel planning system that demonstrates advanced patterns:

- **Architecture patterns**: Specialized roles, LLM contexts, tool workflows
- **Observability features**: Automatic instrumentation, agent identification, conversation tracking
- **Real-world application**: How this scales to enterprise systems with many agents

In [ ]:
import uuid
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
from langgraph.prebuilt import create_react_agent
from fiddler_langgraph.tracing.instrumentation import (
    LangGraphInstrumentor,
    set_conversation_id,
    set_llm_context,
)

# Initialize instrumentation
instrumentor = LangGraphInstrumentor(fdl_client)
instrumentor.instrument()

print("✅ LangGraph instrumentation activated")

# Business logic tools with observability
@tool
def search_flights(origin: str, destination: str, date: str) -> str:
    """Search for available flights with pricing information."""
    flight_id = f"FL{uuid4().hex[:8].upper()}"
    return f"Found flight {flight_id}: {origin} to {destination} on {date}, $450"


@tool
def search_hotels(city: str, check_in: str, checkout: str) -> str:
    """Search for available hotels with amenities."""
    hotel_id = f"HT{uuid4().hex[:8].upper()}"
    return f"Found hotel {hotel_id}: Grand Plaza in {city}, {check_in} to {checkout}, $220/night"

# Create specialized agents with distinct contexts
flight_model = ChatOpenAI(model="gpt-4o-mini", temperature=0.2)
set_llm_context(flight_model, "flight-booking-specialist-premium")

hotel_model = ChatOpenAI(model="gpt-4o-mini", temperature=0.2)
set_llm_context(hotel_model, "hotel-booking-specialist-concierge")

# Create agents
flight_agent = create_react_agent(
    model=flight_model,
    tools=[search_flights],
    prompt="You are a flight booking specialist. Find optimal flight options."
)

hotel_agent = create_react_agent(
    model=hotel_model,
    tools=[search_hotels],
    prompt="You are a hotel booking specialist. Find great accommodations."
)

print("✅ Specialized agents created with unique LLM contexts")
print("📊 Each agent will appear with distinct labels in Fiddler dashboard")

## Part 3: Session Management and Business Context

* Critical learning: What SDK handles automatically vs. what needs custom code
* Clear division: Technical metrics (SDK) vs. business context (custom)
* Best practices: Don't duplicate SDK functionality, focus on business value
* Key insight: Leverage automation, enhance with domain expertise

The Fiddler LangGraph SDK automatically captures:
* Agent execution traces and performance metrics
* LLM interactions, token usage, and response times
* Tool invocations and error handling
* Conversation flows through callback instrumentation

This session manager focuses on **business-specific context** that the SDK cannot infer.

In [ ]:
from datetime import datetime

class BusinessSessionManager:
    """
    Simplified session manager that focuses on business context
    rather than duplicating SDK automatic collection.
    """
    
    def __init__(self):
        self.sessions = {}
    
    def start_session(self, user_id: str, user_type: str, business_context: dict = None) -> str:
        """
        Start session with business context that Fiddler SDK cannot auto-detect.
        The SDK will automatically handle conversation tracking via set_conversation_id().
        """
        session_id = f'session_{user_id}_{uuid4().hex[:8]}'
        
        # Set conversation ID for automatic SDK tracking
        set_conversation_id(flight_agent, session_id)
        set_conversation_id(hotel_agent, session_id)
        
        # Store only business context the SDK cannot infer
        self.sessions[session_id] = {
            'user_id': user_id,
            'user_type': user_type,  # Business classification
            'start_time': datetime.now(),
            'business_context': business_context or {},  # Budget, preferences, etc.
            # Note: Request counts, success rates, timing, etc. are automatically
            # captured by the Fiddler SDK through LangGraph instrumentation
        }
        
        print(f'📱 Started session {session_id}')
        print(f'🔍 SDK will automatically track: execution traces, performance, conversations')
        print(f'📊 Session manager tracks: business context, user classification')
        return session_id
    
    def process_request(self, session_id: str, request: str, agent_type: str = 'flight'):
        """
        Process request with business context injection.
        SDK automatically captures: timing, success/failure, LLM interactions, tool calls.
        """
        if session_id not in self.sessions:
            return {'error': 'Session not found'}
        
        session = self.sessions[session_id]
        
        # Choose agent (SDK will automatically track which agent is used)
        agent = flight_agent if agent_type == 'flight' else hotel_agent
        
        try:
            # Add business context to the request if needed
            enhanced_request = request
            if session['business_context']:
                context_info = f' (User type: {session['user_type']})'
                enhanced_request += context_info
            
            # Execute agent - SDK automatically captures everything
            result = agent.invoke({
                'messages': [{'role': 'user', 'content': enhanced_request}]
            })
            
            # SDK automatically captures: execution time, success, LLM tokens,
            # tool calls, conversation flow, error handling, etc.
            
            return {'success': True, 'result': result, 'session_id': session_id}
            
        except Exception as e:
            # SDK automatically captures error details and context
            print(f'Request failed (SDK captured error details): {e}')
            return {'success': False, 'error': str(e)}
    
    def get_business_context(self, session_id: str):
        """
        Get business context. For technical metrics (performance, success rates,
        execution times, etc.), check the Fiddler dashboard - the SDK captures all of that.
        """
        if session_id not in self.sessions:
            return {'error': 'Session not found'}
        
        session = self.sessions[session_id]
        duration = datetime.now() - session['start_time']
        
        return {
            'session_id': session_id,
            'user_type': session['user_type'],
            'business_context': session['business_context'],
            'session_duration_minutes': round(duration.total_seconds() / 60, 2),
            'note': 'For performance metrics, success rates, and technical details, check Fiddler dashboard'
        }

# Initialize simplified session manager
session_manager = BusinessSessionManager()

print('✅ Business session manager ready')
print('Division of responsibilities:')
print('🤖 Fiddler SDK automatically captures:')
print('   - Agent execution traces and span hierarchy')
print('   - LLM interactions, tokens, response times')
print('   - Tool invocations and parameters')
print('   - Success/failure rates and error details')
print('   - Conversation flow and continuity')
print('📊 Session manager adds:')
print('   - Business user classification')
print('   - Custom business context and preferences')
print('   - Domain-specific metadata')

## Part 4: Real-World Scenarios

Part 4: Real-World Scenarios

* **Scenario design:** Why these specific examples matter
* **Dashboard expectations**: What users will see in Fiddler
* **Learning objectives**: Business context enhancement, user segmentation value
* **Production relevance**: Patterns that scale to real applications

Let's test our system with realistic travel planning scenarios.

In [ ]:
# Scenario 1: Business Travel with Rich Business Context
print('🚀 Scenario 1: Executive Business Travel')
print('=' * 50)

# Business context that SDK cannot auto-detect
business_context = {
    'budget_category': 'corporate',
    'priority_level': 'high',
    'travel_class': 'business',
    'expense_code': 'CORP-2024-Q1',
    'approval_status': 'pre_approved'
}

business_session = session_manager.start_session(
    user_id='exec_user_001',
    user_type='business_premium',
    business_context=business_context
)

# SDK automatically captures execution details, timing, success/failure
flight_result = session_manager.process_request(
    session_id=business_session,
    request='Book urgent business class flight from NYC to London for Monday',
    agent_type='flight'
)
print(f'✅ Flight request: {flight_result['success']} (details in Fiddler dashboard)')

hotel_result = session_manager.process_request(
    session_id=business_session,
    request='Find 5-star hotel near London financial district for 2 nights',
    agent_type='hotel'
)
print(f'✅ Hotel request: {hotel_result['success']} (details in Fiddler dashboard)')

# Get business context (technical metrics are in Fiddler dashboard)
business_info = session_manager.get_business_context(business_session)
print(f'📊 Business Context:')
print(f'  User Type: {business_info['user_type']}')
print(f'  Budget Category: {business_info['business_context']['budget_category']}')
print(f'  Session Duration: {business_info['session_duration_minutes']} minutes')
print(f'  Note: {business_info['note']}')

In [ ]:
# Scenario 2: Family Vacation with Different Business Context
print('🏖️ Scenario 2: Family Vacation Planning')
print('=' * 50)

family_context = {
    'budget_category': 'personal',
    'group_size': 4,
    'traveler_ages': [35, 33, 8, 5],
    'special_requirements': ['child_meals', 'family_rooms'],
    'price_sensitivity': 'high'
}

family_session = session_manager.start_session(
    user_id='family_user_002',
    user_type='leisure',
    business_context=family_context
)

# SDK tracks all technical details automatically
family_flight = session_manager.process_request(
    session_id=family_session,
    request='Find family-friendly flights from LAX to Orlando for 4 people',
    agent_type='flight'
)

family_hotel = session_manager.process_request(
    session_id=family_session,
    request='Family hotel with pool and kids activities near theme parks',
    agent_type='hotel'
)

family_info = session_manager.get_business_context(family_session)
print(f'📊 Family Session Business Context:')
print(f'  User Type: {family_info['user_type']}')
print(f'  Group Size: {family_info['business_context']['group_size']} people')
print(f'  Price Sensitivity: {family_info['business_context']['price_sensitivity']}')
print(f'💡 Key Insight: Fiddler SDK automatically captured all execution details,     while session manager focuses on business context differentiation.')

## Part 5: Analytics and Dashboard Review

* **Data inventory**: What the notebook generated
* **Dashboard guidance**: Specific views and metrics to explore
* **Production insights**: Performance benchmarks, scalability, monitoring
* **Key takeaway**: Focus on business logic, leverage SDK automation

Let's review the comprehensive data we've generated.

In [ ]:
# Simplified analytics focusing on business insights
print('📊 BUSINESS ANALYTICS SUMMARY')
print('=' * 60)

# Business context comparison
total_sessions = len(session_manager.sessions)
print(f'🏗️ SESSION OVERVIEW')
print(f'   Total Sessions: {total_sessions}')

print(f'👥 BUSINESS CONTEXT BREAKDOWN')
print('-' * 40)

for session_id, session in session_manager.sessions.items():
    context = session_manager.get_business_context(session_id)
    print(f'Session {session['user_id']}:')
    print(f'  User Type: {context['user_type']}')
    if 'budget_category' in context['business_context']:
        print(f'  Budget: {context['business_context']['budget_category']}')
    if 'group_size' in context['business_context']:
        print(f'  Group Size: {context['business_context']['group_size']}')
    print(f'  Duration: {context['session_duration_minutes']} min')
    print()

print(f'🎯 FIDDLER SDK AUTOMATIC COLLECTION')
print('=' * 60)
print('The Fiddler LangGraph SDK automatically captured:')
print('')
print('📈 TECHNICAL METRICS (check Fiddler dashboard):')
print('  ✅ Agent execution traces with parent-child relationships')
print('  ✅ LLM interactions: prompts, responses, token usage, timing')
print('  ✅ Tool invocations: parameters, results, execution time')
print('  ✅ Success/failure rates and error details')
print('  ✅ Conversation continuity through conversation IDs')
print('  ✅ Agent-specific contexts (flight-specialist vs hotel-specialist)')
print('')
print('🏢 BUSINESS VALUE-ADD (session manager contribution):')
print('  📊 User segmentation (business vs leisure)')
print('  💰 Budget classification and preferences')
print('  👥 Group composition and special requirements')
print('  🎯 Business context for enhanced dashboard filtering')

print(f'🚀 DASHBOARD RECOMMENDATIONS')
print('=' * 60)
print('In your Fiddler dashboard, create views by:')
print('1. **Agent Type**: Compare flight-specialist vs hotel-specialist performance')
print('2. **Conversation ID**: See complete user journeys across agents')
print('3. **LLM Context**: Analyze specialized agent behavior patterns')
print('4. **User Segments**: Filter by business vs leisure travel patterns')
print('5. **Error Patterns**: Identify common failure points across agents')

print(f'💡 KEY INSIGHT')
print('=' * 60)
print('The Fiddler LangGraph SDK\'s automatic instrumentation eliminates')
print('the need for manual metrics collection. Focus your custom code on:')
print('• Business context the SDK cannot infer')
print('• Domain-specific classifications')
print('• Custom business logic and rules')
print('Let the SDK handle the technical observability automatically!')

## Summary

Congratulations! You've completed the advanced Fiddler LangGraph SDK tutorial. This notebook demonstrated:

### 🏗️ **Advanced Patterns Covered**
- **Multi-agent systems** with specialized roles and contexts
- **Performance optimization** with environment-specific configurations
- **Session management** with comprehensive tracking
- **Business intelligence** integration with custom metrics
- **Real-world scenarios** with practical applications

### 📊 **Observability Features**
- **Agent-specific contexts** for detailed dashboard segmentation
- **Conversation tracking** across multi-turn interactions
- **Performance metrics** and success rate monitoring
- **User segmentation** and behavior analysis
- **Business outcome tracking** with actionable insights

### 🎯 **Production Readiness**
- **Environment-specific configurations** for dev/staging/production
- **Resource optimization** with appropriate span limits and sampling
- **Error handling** and comprehensive logging
- **Scalable architecture** patterns for enterprise deployment

Your Fiddler dashboard now contains rich observability data showcasing enterprise-grade agentic monitoring. Use these patterns as a foundation for building production-ready AI applications.

---

**Resources:**
- [Fiddler LangGraph SDK Documentation](../technical-reference/fiddler-langgraph-sdk.md)
- [Quick Start Guide](../quick-start-guide.md)
- [Getting Started with Agentic Monitoring](../getting-started-with-agentic-monitoring.md)
- [Support and Feedback](mailto:help@fiddler.ai)